In [1]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from ipywidgets import widgets
%matplotlib widget

def mask_overlay(image, mask, color=(0., 1., 0.), weight=.8):
    """
    Helper function to visualize mask on the top of the aneurysm
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    mask = mask.astype(np.uint8)
    weighted_sum = image * weight + mask * (1 - weight) # cv2.addWeighted(image, 1 - weight, mask, weight, 0.,  dtype=cv2.CV_32F)
    img = image.copy()
    ind = mask[:, :, 1] > 0
    img[ind] = weighted_sum[ind]
    return img


class ImageSlicer(object):

    def __init__(self, ax, img, mask, start_index):
        self.ax = ax

        # convert to numpy array
        self.image_np = img
        self.mask_np = mask

        # get number of slices
        _, _, self.slices = self.image_np.shape
        self.ind = start_index if start_index is not None else self.slices // 2
        self.show_mask = True

        # plot image with mask overlay
        self.image_plt = self.ax.imshow(self.overlay)
        self._draw()

    @property
    def overlay(self):
        # get image and mask slice
        image = self.image_np[:, :, self.ind]
        image = image / np.max(image)
        image = np.dstack((image, image, image))
        mask = self.mask_np[:, :, self.ind]

        # create masked overlay
        if self.show_mask:
            return mask_overlay(image, mask)
        else: 
            return image

    def onscroll(self, event):

        # get new slice number
        self.ind = event['new']
        self.update()

    def on_show_mask(self, event):

        self.show_mask = event['new']
        with open("file.txt", "a") as f:
            f.write(f"{self.show_mask}\n")
        self.update()

    def update(self):

        # draw overlay
        self.image_plt.set_data(self.overlay)
        self._draw()

    def _draw(self):
        try:
            self.image_plt.axes.figure.canvas.draw()
        except Exception as e:
            with open("file.txt", "a") as f:
                f.write(f"Exception {e}\n")
            pass


def plot3d(img, mask, start_index = None):

    #
    figure, ax = plt.subplots(1, 1)
    tracker = ImageSlicer(ax, img, mask, start_index)

    #
    int_slider = widgets.IntSlider(
        value=tracker.ind,
        min=0,
        max=tracker.slices,
        step=1,
        description='Slice',
        continuous_update=True
    )
    int_slider.observe(tracker.onscroll, 'value')

    show_mask = widgets.Checkbox(
        value=True,
        description='Show Mask',
        disabled=False,
        indent=False
    )
    show_mask.observe(tracker.on_show_mask, 'value')

    return figure, int_slider, show_mask



In [2]:
import yaml
import os
from pytorch3dunet.augment.transforms import Transformer

config_path = "./train-configs/aug_tests/id.yml"

config = yaml.safe_load(open(config_path, 'r'))

transform_config = config["loaders"]["train"]["transformer"]
datapath = config["loaders"]["train"]["file_paths"][0]

transformer = Transformer(transform_config, {})
raw_transform = transformer.raw_transform()
label_transform = transformer.label_transform()

files = os.listdir("./data/train")
print(files)

['A024.h5', 'A096_R.h5', 'A094_R.h5', 'A054.h5', 'A121.h5', 'A088.h5', 'A093.h5', 'A009.h5', 'A091_R.h5', 'A126.h5', 'A119.h5', 'A083.h5', 'A100.h5', 'A098.h5', 'A057.h5', 'A010.h5', 'A130_L.h5', 'A124.h5', 'A046.h5', 'A038_M.h5', 'A130_R.h5', 'A078_L.h5', 'A059_L.h5', 'A035.h5', 'A042.h5', 'A062_L.h5', 'A095.h5', 'A068.h5', 'A040.h5', 'A108.h5', 'A105_L.h5', 'A025.h5', 'A138.h5', 'A127.h5', 'A086.h5', 'A045.h5', 'A112.h5', 'A060.h5', 'A076.h5', 'A026.h5', 'A017_L.h5', 'A021.h5', 'A013.h5', 'A005.h5', 'A082.h5', 'A071.h5', 'A136.h5', 'A006.h5', 'A038_R.h5', 'A067.h5', 'A115.h5', 'A087.h5', 'A079.h5', 'A074.h5', 'A015.h5', 'A001.h5', 'A135.h5', 'A081.h5', 'PA5.h5', 'A123.h5', 'A064.h5', 'PA6.h5', 'A073.h5', 'A092.h5', 'A113.h5', 'A023_R.h5', 'A114.h5', 'A120.h5', 'A038_L.h5', 'A041.h5', 'A016.h5', 'A089_R.h5', 'A044.h5', 'A129.h5', 'A050.h5']


In [98]:
from scipy.ndimage import percentile_filter, binary_dilation

padding = 40
def pad_min_max(max, ival):
    return slice(
        np.clip(ival[0] - padding, a_min=0, a_max=max - 1),
        np.clip(ival[1] + padding, a_min=0, a_max=max - 1)
    )

def get_aneurysm_bounds(mask):
    x_s = mask.sum(axis=(1, 2))
    y_s = mask.sum(axis=(0, 2))
    z_s = mask.sum(axis=(0, 1))

    x = np.where(x_s)[0][[0, -1]]
    y = np.where(y_s)[0][[0, -1]]
    z = np.where(z_s)[0][[0, -1]]

    x = pad_min_max(mask.shape[0], x)
    y = pad_min_max(mask.shape[1], y)
    z = pad_min_max(mask.shape[2], z)

    return x, y, z

def check_percentile(low, high):


    
    return s / len(files)

start_low = 1
start_high = 90
r = 10

res = np.zeros((15, 15))
for file in files:
    print(file)
    with h5py.File(f"./data/train/{file}", "r") as f:
        gt = f["label"][:]
        raw = f["raw"][:]

        bounds = get_aneurysm_bounds(gt)

        gt_bounded = gt[bounds]
        raw_bounded = raw[bounds]

        ignore_mask = np.logical_not(binary_dilation(gt_bounded, iterations=5))
        gt_bounded[ignore_mask] = 0

        for low in range(start_low, start_low + r):
            for high in range(start_high, start_high + r):
                print(low, high)
                low_perc = np.percentile(raw, low)
                high_perc = np.percentile(raw, high)
                
                raw_copy = raw_bounded.copy()
                raw_copy[raw_copy < low_perc] = 0
                raw_copy[raw_copy > high_perc] = 1
                raw_copy[ignore_mask] = 0

                res[low - start_low, high - start_high] += np.mean(np.abs(gt_bounded - raw_copy))
    

        
res

A024.h5
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
2 90
2 91
2 92
2 93
2 94
2 95
2 96
2 97
2 98
2 99
3 90
3 91
3 92
3 93
3 94
3 95
3 96
3 97
3 98
3 99
4 90
4 91
4 92
4 93
4 94
4 95
4 96
4 97
4 98
4 99
5 90
5 91
5 92
5 93
5 94
5 95
5 96
5 97
5 98
5 99
6 90
6 91
6 92
6 93
6 94
6 95
6 96
6 97
6 98
6 99
7 90
7 91
7 92
7 93
7 94
7 95
7 96
7 97
7 98
7 99
8 90
8 91
8 92
8 93
8 94
8 95
8 96
8 97
8 98
8 99
9 90
9 91
9 92
9 93
9 94
9 95
9 96
9 97
9 98
9 99
10 90
10 91
10 92
10 93
10 94
10 95
10 96
10 97
10 98
10 99
A096_R.h5
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
2 90
2 91
2 92
2 93
2 94
2 95
2 96
2 97
2 98
2 99
3 90
3 91
3 92


KeyboardInterrupt: 

In [ ]:
res[:10, :10].argmin()
t = res[:10, :10]
m = np.unravel_index(t.argmin(), t.shape)
m[0] + start_low, m[1] + start_high

(10, 99)

In [22]:
padding = 50
def pad_min_max(max, ival):
    return slice(
        np.clip(ival[0] - padding, a_min=0, a_max=max - 1),
        np.clip(ival[1] + padding, a_min=0, a_max=max - 1)
    )

def get_aneurysm_bounds(mask):
    x_s = mask.sum(axis=(1, 2))
    y_s = mask.sum(axis=(0, 2))
    z_s = mask.sum(axis=(0, 1))

    x = np.where(x_s)[0][[0, -1]]
    y = np.where(y_s)[0][[0, -1]]
    z = np.where(z_s)[0][[0, -1]]

    x = pad_min_max(mask.shape[0], x)
    y = pad_min_max(mask.shape[1], y)
    z = pad_min_max(mask.shape[2], z)

    return x, y, z

In [20]:
import torch
f = h5py.File("./test/2800_0.h5", "r")
gt = f["label"][0, 0]
pred = f["pred"][0, 0]
raw = f["raw"][0, 0]


#pred = (torch.sigmoid(torch.from_numpy(pred)) > 0.5).long()


fig, slider = plot3d(raw, pred)
slider

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=64, description='Slice', max=128)

In [16]:
from scipy.ndimage import percentile_filter, binary_dilation, maximum_filter, minimum_filter, generic_filter
import pandas as pd
import torch.nn as nn
import torch
i = 0

def load_case(i):
    f = h5py.File(f"./data/train/{files[i]}", "r")
    return f["raw"][:], f["label"][:]

# img, mask = load_case(i)
# img /= img.max()

# img_t = raw_transform(img)
# mask_t = label_transform(mask)

# img = img[:128, :128, 128]
# mask = mask[128, :128, 128]

# out_img = percentile_filter(img, size=3, percentile=50)

def transform(img, mask):
    high_perc = np.percentile(img, 99)
    print(high_perc)
    img_b = img.copy()
    img_b[img_b >= high_perc] = 1
    img_b[img_b < high_perc] = 0
    img_b = minimum_filter(img_b, size=2)
    img_b = binary_dilation(img_b, iterations=2)

    # img[mask_e != 1] = 0

    return img_b, mask

def calc_overlap(img_b, mask):
    return np.mean(img_b[mask == 1])

# ret = []
# ret_2 = []

# for i in range(len(files)):
#     img, mask = load_case(i)
#     img /= img.max()
#     img_b, mask = transform(img, mask)
#     ret.append(calc_overlap(img_b, mask))
#     ret_2.append(img_b.sum() / (img_b.shape[0] * img_b.shape[1] * img_b.shape[2]))

# s = pd.Series(ret)
# s_2 = pd.Series(ret_2)
# print(s.describe())
# print(s_2.describe())

i = 0
img, mask = load_case(i)
#img_b, mask = transform(img, mask)

# bounds = get_aneurysm_bounds(mask)

# zero_mask = np.zeros(img_b.shape)
# print(calc_overlap(img_b, mask))

pool = nn.MaxPool3d((2, 2, 2))
img = pool(torch.from_numpy(img[None, None]))[0, 0].numpy()
mask = pool(torch.from_numpy(mask[None, None]))[0, 0].numpy()

fig, slider, show_mask = plot3d(mask, mask)

widgets.Box(
    [slider, show_mask]
)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/lm/anaconda3/envs/mlmed/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide


Box(children=(IntSlider(value=64, description='Slice', max=128), Checkbox(value=True, description='Show Mask',…

In [ ]:
Coverage 1 weniger
count    75.000000
mean      0.850895
std       0.067300
min       0.658271
25%       0.818405
50%       0.867841
75%       0.898143
max       0.942187

Coverage
count    75.000000
mean      0.997276
std       0.008810
min       0.931575
25%       0.997698
50%       0.999767
75%       1.000000
max       1.000000

% of input
dtype: float64
count    75.000000
mean      0.020444
std       0.003271
min       0.009217
25%       0.018929
50%       0.020931
75%       0.022494
max       0.025825
dtype: float64

In [19]:
from scipy.ndimage import percentile_filter, binary_dilation, binary_erosion, maximum_filter, minimum_filter, generic_filter
import pandas as pd
i = 0

img, mask = load_case(i)
img /= img.max()

ret = []
for i in range(len(files)):
    print(f"{i}/{len(files)}")
    img, mask = load_case(i)

    mask = binary_erosion(mask, iterations=3)


    t = img[mask == 1]
    ret.append([t.min(), t.max(), len(t), np.std(t), np.mean(t), t.sum() / (t.max() * len(t))])
    

s_3 = pd.DataFrame(ret, columns=["min", "max", "len", "std", "mean", "ratio"])

0/75
1/75
2/75
3/75
4/75
5/75
6/75
7/75
8/75
9/75
10/75
11/75
12/75
13/75
14/75
15/75
16/75
17/75
18/75
19/75
20/75
21/75
22/75
23/75
24/75
25/75
26/75
27/75
28/75
29/75
30/75
31/75
32/75
33/75
34/75
35/75
36/75


ValueError: zero-size array to reduction operation minimum which has no identity

In [13]:
s.describe()

,min,max,len,std,mean,ratio
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,0.012214,0.708982,3614.773333,0.173980,0.346963,0.490403
std,0.021685,0.127396,7128.514000,0.042325,0.072827,0.062249
min,0.000000,0.331886,159.000000,0.047812,0.121667,0.337518
25%,0.000000,0.624898,637.000000,0.145646,0.303383,0.454846
50%,0.000000,0.706570,1247.000000,0.176744,0.341383,0.489148
75%,0.021395,0.775113,3548.500000,0.205331,0.389242,0.529372
max,0.093032,0.942536,41464.000000,0.280563,0.536797,0.667593


In [18]:
s_2.describe()

,min,max,len,std,mean,ratio
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,0.311028,0.703913,2179.920000,0.073762,0.519283,0.737684
std,0.127894,0.128716,5467.067003,0.029169,0.118223,0.117143
min,0.023478,0.331886,9.000000,0.019481,0.131613,0.396561
25%,0.214704,0.622517,143.000000,0.053781,0.454717,0.654247
50%,0.323343,0.703904,439.000000,0.069557,0.530370,0.758229
75%,0.391739,0.775113,1709.000000,0.087093,0.604220,0.833005
max,0.645861,0.942536,31728.000000,0.206786,0.752791,0.957274
